# MPC4quantum: A Guide to Robust Quantum State Preparation

This notebook provides a complete, step-by-step guide to setting up and running the **MPC4quantum** library in Google Colab. We will handle all necessary installations and then run a simulation to prepare a quantum NOT gate.

[Image of a diagram demonstrating the receding horizon optimizations used in model predictive control.]

**The Goal:** We will use Model Predictive Control (MPC) to find the optimal control pulses to transform a qubit from the state $$|0\rangle$$ to the state $$|1\rangle$$.

## Step 1: Upload Your Project Files

Before running any code, you must upload the project files to this Colab environment.

1.  Click the **folder icon** on the left sidebar.
2.  Drag and drop the entire `mpc4quantum` project folder into the file explorer.

Your file structure should look like this:

In [1]:
import numpy as np
import qutip as qt
from mpc4quantum import QExperiment

def blackman(ts, t0, tf, dt):
    """
    Evaluate Blackman pulse to resolution dt using 1D linear interpolation.
    """
    M = int((tf - t0) / dt)
    t_interp = np.linspace(t0, tf, M)
    f_interp = np.blackman(M)
    return np.interp(ts, t_interp, f_interp, left=0, right=0)

class RWA_Qubit:
    def __init__(self, wQ, wD, wR):
        """
        Wrapper for qubit simulations in a rotating frame after the rotating wave approximation.

        :param wQ: Qubit frequency
        :param wD: Drive frequency
        :param wR: Rotating wave frequency
        """
        self.dim_s = 2
        self.dim_x = self.dim_s ** 2
        self.dim_u = 1

        self._w0 = wQ
        self._wD = wD
        self._wR = wR

        H0 = 1 / 2 * (self._w0 - self._wR) * qt.sigmaz()
        H1 = 1 / 2 * qt.sigmax()
        self.H_list = [H0, H1]
        self.QE = QExperiment(H0, [H1])

    def u1(self, ts, args):
        # Blackman pulse (truncated Gaussian) with modifications for the rotating frame
        return args['A'] * blackman(ts, args['t0'], args['tf'], args['dt']) * np.cos((self._wD - self._wR) * ts)

In [2]:
import mpc4quantum as m4q
from mpc4quantum.linearize import create_library
import numpy as np
import qutip as qt
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import qutip_qip
from qutip_qip import operations

# To ensure the playground directory exists for saving plots
if not os.path.exists('./playground'):
    os.makedirs('./playground')


# Default args for plot_operator
imshow_args = {'norm': mpl.colors.SymLogNorm(vmin=-1, vmax=1, linthresh=1e-3), 'cmap': plt.get_cmap('RdBu_r')}
cmap = plt.get_cmap('tab10')
rootname = '2021_09_13_OSQP'


for order in range(1, 2): # In the original test this loops from 1 to 4
    # Parameters
    # ==========
    sat = 1
    du = .25
    clock = m4q.StepClock(dt=0.05, horizon=15, n_steps=50)

    # Experiment
    # ==========
    freqs = {'wQ': np.pi, 'wD': np.pi, 'wR': np.pi}
    qubit = RWA_Qubit(**freqs)
    gate_synth = m4q.QSynthesis(qubit.H_list[0], [qubit.H_list[1]])

    # Model
    # =====
    dim_s = qubit.dim_s
    As_cts_list = [-1j * (qt.tensor(h, qt.identity(dim_s)) - qt.tensor(qt.identity(dim_s), h.trans()))
                   for h in qubit.H_list]
    As_cts_list = [qt.operator_to_vector(op).full() for op in As_cts_list]


    # DMDc object
    dim_lift = len(create_library(order, qubit.dim_u)[1:])
    dim_process = qubit.dim_x ** 2
    model1 = m4q.DMDc(dim_process, dim_process, dim_process * dim_lift, A_init)

    # Operations
    Rx = qutip_qip.operations.rx(1e-3)
    U0 = Rx * qt.identity(2)
    Uf = qt.sigmax()
    p0 = qt.tensor(U0, U0.conj()).full().flatten()
    pf = qt.tensor(Uf, Uf.conj()).full().flatten()

    # Benchmarks
    # ==========
    P_bm = np.hstack([pf.reshape(-1, 1)] * (clock.horizon + 1))
    U_bm = np.hstack([np.ones([qubit.dim_u, 1]) * .5] * clock.horizon)

    # Cost
    # ====
    Q = np.identity(len(p0))
    Qf = Q * 1e1
    R = np.identity(qubit.dim_u) * 1e-2

    # MPC
    # ===
    def exit_condition(p2, p1, u1):
        return ((p1 - pf).conj().T @ Q @ (p1 - pf)).real < 1e-2

    data, model2, exit_code = m4q.mpc(p0, qubit.dim_u, order, P_bm, U_bm, clock, gate_synth, model1, Q, R, Qf,
                                      sat=sat, du=du, exit_condition=exit_condition)
    ps, us = data

    # Save diagnostics
    # ================
    path = './playground/{}_NOT_gate/sat_{}_du_{}_{}/'.format(rootname, sat, du, clock.to_string())
    if not os.path.exists(path):
        os.makedirs(path)
    transparent = False
    if np.any(us):
        tsp1 = np.hstack([clock.ts_sim, clock.ts_sim[-1] + clock.dt])

        fig, ax = plt.subplots(1)
        ax.step(tsp1, np.hstack([us[0], us[0, -1]]), where='post')
        ax.set_ylim(-1.1, 1.1)
        ax.set_title("Control Pulse")
        ax.set_xlabel("Time")
        ax.set_ylabel("Amplitude")
        fig.savefig(path + 'control_order_{}.png'.format(order), transparent=transparent)
        plt.show()


        def plot_state_prediction(psi0, name):
            psi0 = psi0 / psi0.norm()
            rho0 = psi0.proj().full().reshape(-1, 1)
            xs = [None] * ps.shape[1]
            for i, p in enumerate(ps.T):
                xs[i] = p.reshape((2 ** 2, 2 ** 2)) @ rho0
            xs = np.hstack(xs)
            fig, axes = plt.subplots(1, 4, figsize=[3 * 4 + 3, 3])
            for i in range(4):
                ax = axes[i]
                ax.plot(tsp1, xs[i].real, ls='-', color=cmap(i), label=f'real(ρ_{i})')
                ax.plot(tsp1, xs[i].imag, ls='--', color=cmap(i), label=f'imag(ρ_{i})')
                ax.set_ylim([-1.1, 1.1])
                ax.legend()
            fig.tight_layout()
            fig.savefig(path + '{}_order_{}.png'.format(name, order), transparent=transparent)
            plt.show()


        # Example state preparations based on gate
        state = qt.basis(2, 0)
        plot_state_prediction(state, '1psi0')
        state = -3 * qt.basis(2, 1) + qt.basis(2, 0)
        plot_state_prediction(state, 'n3psi1_1psi0')

        fig, ax = plt.subplots(1)
        ax.plot(tsp1, [((p - pf).conj().T @ Q @ (p - pf)).real for p in ps.T])
        ax.set_ylim([1e-3, 1e1])
        ax.set_yscale('log')
        ax.set_title("Cost Function vs. Time")
        ax.set_xlabel("Time")
        ax.set_ylabel("Cost")
        fig.savefig(path + 'cost_order_{}.png'.format(order), transparent=transparent)
        plt.show()

NameError: name 'A_init' is not defined